# Notebook 2 - SQL avec vraies bases de données
## Analyse e-commerce avec PostgreSQL en ligne




### 🎯 Objectifs pédagogiques
- Connecter Python à une vraie base de données PostgreSQL
- Écrire des requêtes SQL complexes sur des données réelles
- Implémenter des analyses RFM avec SQL
- Intégrer SQL et pandas pour des analyses avancées
- Gérer les connexions et la sécurité

### 🛍️ Contexte du projet
Vous analysez les données d'un vrai dataset e-commerce (Brazilian E-Commerce Public Dataset) hébergé sur une base PostgreSQL.

Objectif : créer une segmentation clientèle pour optimiser les campagnes marketing.


## Partie 1 : Connexion à la base de données réelle

### 🔧 Installation et configuration


# Installation des dépendances


```
pip install psycopg2-binary sqlalchemy pandas python-dotenv
```




In [27]:
import psycopg2
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

import os
from dotenv import load_dotenv

from sqlalchemy import create_engine, text

### 🌐 Base de données PostgreSQL gratuite (ElephantSQL)

**Option 1 : ElephantSQL (20MB gratuit)**
1. Créez un compte sur [elephantsql.com](https://www.elephantsql.com/)
2. Créez une instance "Tiny Turtle" (gratuite)
3. Récupérez vos credentials

**Option 2 : Supabase (500MB gratuit)**
1. Créez un compte sur [supabase.com](https://supabase.com/)
2. Créez un nouveau projet
3. Récupérez l'URL de connexion PostgreSQL

In [28]:
# Configuration de connexion (à adapter selon votre provider)
DATABASE_CONFIG = {
    'host': 'aws-0-eu-west-3.pooler.supabase.com',  # Ou votre host Supabase
    'database': os.getenv("DATABASE"),
    'user': os.getenv("USER_postgres"),
    'password': os.getenv("PASSWORD"),
    'port': os.getenv("PORT"),
}

# Création de l'engine SQLAlchemy
engine = create_engine(
    f"postgresql://{DATABASE_CONFIG['user']}:{DATABASE_CONFIG['password']}@"
    f"{DATABASE_CONFIG['host']}:{DATABASE_CONFIG['port']}/{DATABASE_CONFIG['database']}"
)

# Test de connexion
def test_connection():
    """
    Testez votre connexion à la base

    Étapes :
    1. Utilisez pd.read_sql() pour exécuter "SELECT version()"
    2. Affichez la version PostgreSQL
    3. Gérez les erreurs de connexion
    """
    try:
        # Votre code ici
        version = pd.read_sql("SELECT version();", engine)
        print(version)
        pass
    except Exception as e:
        print(f"Erreur de connexion : {e}")
        return False
    return True

test_connection()

                                             version
0  PostgreSQL 17.4 on aarch64-unknown-linux-gnu, ...


True


## Partie 2 : Import du dataset e-commerce

### 📊 Dataset Brazilian E-Commerce
Nous utilisons le célèbre dataset Olist (100k commandes réelles).

**Tables à créer :**
1. **customers** : customer_id, customer_city, customer_state
2. **orders** : order_id, customer_id, order_status, order_date, order_delivered_date
3. **order_items** : order_id, product_id, seller_id, price, freight_value
4. **products** : product_id, product_category, product_weight_g
5. **sellers** : seller_id, seller_city, seller_state

In [29]:
### 📥 Import des données via API

import requests
import zipfile
import io
import pandas as pd

def download_olist_dataset():

    """
    Télécharge le dataset Olist depuis Kaggle API

    Alternative : Utilisez l'API publique de l'IBGE (Institut brésilien)
    pour des données e-commerce synthétiques mais réalistes
    """

    # URL des données publiques brésiliennes
    IBGE_API = "https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce"

    # Récupération des données de villes (pour la géolocalisation)
    cities_url = f"{IBGE_API}localidades/municipios"

    try:
        response = requests.get(cities_url)
        cities_data = response.json()

        # Convertir en DataFrame
        cities_df = pd.DataFrame(cities_data)

        # Votre code pour nettoyer et structurer
        # Créez des données e-commerce réalistes basées sur ces villes

        return cities_df
    except Exception as e:
        print(f"Erreur API IBGE : {e}")
        return None

# Génération de données e-commerce réalistes
def generate_ecommerce_data(cities_df, n_customers=10000):
    """
    Génère des données e-commerce réalistes

    Étapes guidées :
    1. Sélectionnez 50 villes brésiliennes aléatoirement
    2. Créez des clients avec distribution réaliste
    3. Générez des commandes avec saisonnalité
    4. Ajoutez des produits avec catégories cohérentes
    5. Calculez des prix et frais de port basés sur la distance
    """
    pass

In [30]:
customers_clean = pd.read_csv("data/olist_customers_dataset.csv")
customers_clean = customers_clean[['customer_id', 'customer_city', 'customer_state']]

In [31]:
orders_clean = pd.read_csv("data/olist_orders_dataset.csv")
orders_clean = orders_clean[['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_delivered_customer_date']].rename(columns={
        'order_purchase_timestamp': 'order_date',
        'order_delivered_customer_date': 'order_delivered_date'})

print("Max value:", orders_clean['order_date'].max())
print("Min value:", orders_clean['order_date'].min())



Max value: 2018-10-17 17:30:18
Min value: 2016-09-04 21:15:19


In [32]:
order_items_clean = pd.read_csv("data/olist_order_items_dataset.csv")
order_items_clean = order_items_clean[['order_id', 'product_id', 'seller_id', 'price', 'freight_value']]

In [33]:
sellers_clean = pd.read_csv("data/olist_sellers_dataset.csv")
sellers_clean = sellers_clean[['seller_id', 'seller_city', 'seller_state']]

In [34]:
products_clean = pd.read_csv("data/olist_products_dataset.csv")
products_clean = products_clean[['product_id', 'product_category_name', 'product_weight_g']].rename(columns={'product_category_name': 'product_category'})

In [35]:
def create_tables(engine):
    from sqlalchemy import text

    create_customers = text("""
    CREATE TABLE IF NOT EXISTS customers (
        customer_id VARCHAR PRIMARY KEY,
        customer_city VARCHAR(100),
        customer_state CHAR(2)
    );
    """)

    create_sellers = text("""
    CREATE TABLE IF NOT EXISTS sellers (
        seller_id VARCHAR PRIMARY KEY,
        seller_city VARCHAR(100),
        seller_state CHAR(2)
    );
    """)

    create_products = text("""
    CREATE TABLE IF NOT EXISTS products (
        product_id VARCHAR PRIMARY KEY,
        product_category VARCHAR,
        product_weight_g NUMERIC
    );
    """)

    create_orders = text("""
    CREATE TABLE IF NOT EXISTS orders (
        order_id VARCHAR PRIMARY KEY,
        customer_id VARCHAR,
        order_status VARCHAR(50),
        order_date TIMESTAMP,
        order_delivered_date TIMESTAMP,
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    );
    """)

    create_order_items = text("""
    CREATE TABLE IF NOT EXISTS order_items (
        order_id VARCHAR,
        product_id VARCHAR,
        seller_id VARCHAR,
        price NUMERIC,
        freight_value NUMERIC,
        FOREIGN KEY (order_id) REFERENCES orders(order_id),
        FOREIGN KEY (product_id) REFERENCES products(product_id),
        FOREIGN KEY (seller_id) REFERENCES sellers(seller_id)
    );
    """)

    with engine.connect() as conn:
        conn.execute(create_customers)
        conn.execute(create_sellers)
        conn.execute(create_products)
        conn.execute(create_orders)
        conn.execute(create_order_items)
        conn.commit()

In [36]:
create_tables(engine)

In [37]:
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
"""

# Execute query
with engine.connect() as conn:
    tables_df = pd.read_sql(query, conn)

print(tables_df)



    table_name
0    customers
1       orders
2  order_items
3     products
4      sellers


In [38]:
tables = ["customers", "orders", "products", "sellers", "order_items"]

# Exécution de la requête pour chaque table
with engine.connect() as conn:
    for table in tables:
        result = conn.execute(text(f"SELECT COUNT(*) FROM {table}"))
        count = result.scalar()
        print(f"{table} : {count} lignes")


customers : 99441 lignes
orders : 99441 lignes
products : 32951 lignes
sellers : 3095 lignes
order_items : 112650 lignes


In [ ]:
customers_clean.to_sql("customers", engine, if_exists="append", index=False)
print("Donnees inseres dans customers")

In [ ]:
sellers_clean.to_sql("sellers", engine, if_exists="append", index=False)

95

In [ ]:
products_clean.to_sql("products", engine, if_exists="append", index=False)

951

In [ ]:
orders_clean.to_sql("orders", engine, if_exists="append", index=False)
print("Donnees inseres dans orders_clean")


Donnees inseres dans orders_clean


In [ ]:
order_items_clean.to_sql("order_items", engine, if_exists="append", index=False)
print("Donnees inseres dans orders_clean")

Donnees inseres dans orders_clean


## Partie 3 : Requêtes SQL avancées


### 🔍 Analyses SQL à implémenter

#### 1. Analyse RFM (Récence, Fréquence, Montant)
```sql
-- Votre défi : Calculer les métriques RFM pour chaque client
WITH customer_metrics AS (
    SELECT
        c.customer_id,
        c.customer_state,
        -- Récence : jours depuis dernier achat
        DATE_PART('day', DATE '2018-10-17' - MAX(o.order_date)) AS recency,
        -- Fréquence : nombre de commandes
        COUNT(DISCOUNT o.order_id) AS frequency,
        -- Montant : total dépensé
        SUM(oi.price + oi.freight_value) AS monetary
        -- Complétez cette requête CTE
        
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_id, c.customer_state
)

-- Créez les segments RFM (Champions, Loyaux, À risque, etc.)
SELECT
    customer_id,
    customer_state,
    recency_score,
    frequency_score,
    monetary_score,
    CASE
        WHEN recency_score >= 4 AND frequency_score >= 4 THEN 'Champions'
        WHEN recency_score >= 3 AND frequency_score >= 3 THEN 'Loyal Customers'
        -- Ajoutez les autres segments
        WHEN recency_score < 3 AND frequency_score <= 3 THEN 'À Risque'
        ELSE 'Others'
    END as customer_segment
FROM customer_metrics;
```

In [ ]:
query = """
WITH rfm_raw AS (
    SELECT
        c.customer_id,
        -- Récence = jours depuis le dernier achat
        DATE_PART('day', DATE '2018-10-17' - MAX(o.order_date)) AS recency,
        -- Fréquence = nombre de commandes
        COUNT(DISTINCT o.order_id) AS frequency,
        -- Montant = total payé
        SUM(oi.price + oi.freight_value) AS monetary
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_id
),

rfm_scored AS (
    SELECT
        customer_id,
        recency,
        frequency,
        monetary,
        -- Scores entre 1 et 5 avec NTILE
        NTILE(5) OVER (ORDER BY recency ASC) AS recency_score,
        NTILE(5) OVER (ORDER BY frequency DESC) AS frequency_score,
        NTILE(5) OVER (ORDER BY monetary DESC) AS monetary_score
    FROM rfm_raw
)

SELECT
    customer_id,
    recency,
    frequency,
    monetary,
    recency_score,
    frequency_score,
    monetary_score,

    CASE
        WHEN recency_score >= 4 AND frequency_score >= 4 AND monetary_score >= 4 THEN 'Champions'
        WHEN recency_score >= 3 AND frequency_score >= 3 THEN 'Loyal Customers'
        WHEN recency_score >= 4 AND frequency_score BETWEEN 2 AND 3 THEN 'Potential Loyalists'
        WHEN recency_score BETWEEN 2 AND 3 AND frequency_score >= 3 THEN 'At Risk'
        WHEN recency_score <= 2 AND frequency_score <= 2 THEN 'Hibernating'
        WHEN recency_score = 1 THEN 'Lost'
        ELSE 'Others'
    END AS customer_segment

FROM rfm_scored;
"""
# Execute SQL and load results into DataFrame
df = pd.read_sql(query, engine)
print(df.head())

                        customer_id  recency  frequency  monetary  \
0  e60df9449653a95af4549bbfcb18a6eb     48.0          1    510.96   
1  448945bc713d98b6726e82eda6249b9e     48.0          1    497.25   
2  496630b6740bcca28fce9ba50d8a26ef     48.0          1     33.23   
3  e450a297a7bc6839ceb0cf1a2377fa02     48.0          1     73.10   
4  10a79ef2783cae3d8d678e85fde235ac     48.0          1     14.29   

   recency_score  frequency_score  monetary_score customer_segment  
0              1                1               1      Hibernating  
1              1                1               1      Hibernating  
2              1                2               5      Hibernating  
3              1                2               4      Hibernating  
4              1                2               5      Hibernating  


In [58]:
#### 2. Analyse géographique des ventes

def geographic_sales_analysis():
    """
    Analysez les performances par état/région

    Requêtes à écrire :
    1. Top 10 des états par CA
    2. Croissance MoM par région
    3. Taux de conversion par ville
    4. Distance moyenne vendeur-acheteur
    """

    query_top_states = """
        SELECT 
        c.customer_state,
        COUNT(DISTINCT o.order_id) AS num_orders,
        ROUND(SUM(oi.price + oi.freight_value), 2) AS total_revenue,
        ROUND(SUM(oi.price + oi.freight_value) / COUNT(DISTINCT o.order_id), 2) AS avg_basket
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_state
    ORDER BY total_revenue DESC
    LIMIT 10;

    -- Votre requête SQL ici
    -- Utilisez des JOINs et GROUP BY
    -- Calculez le CA, nombre de commandes, panier moyen
    """

    return pd.read_sql(query_top_states, engine)

In [60]:
geographic_sales_analysis()


,customer_state,num_orders,total_revenue,avg_basket
0,SP,40501,5769703.15,142.46
1,RJ,12350,2055401.57,166.43
2,MG,11354,1818891.67,160.20
3,RS,5345,861472.79,161.17
4,PR,4923,781708.80,158.79
5,SC,3546,595127.78,167.83
6,BA,3256,591137.81,181.55
7,DF,2080,346123.35,166.41
8,GO,1957,334212.35,170.78
9,ES,1995,317657.93,159.23


In [71]:
def geographic_sales_mom():
    """
    2. Croissance MoM par état/région
    """
    query_mom_growth = """
    WITH monthly_revenue AS (
        SELECT 
            c.customer_state,
            DATE_TRUNC('month', o.order_date) AS month,
            ROUND(SUM(oi.price + oi.freight_value), 2) AS revenue
        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        JOIN order_items oi ON o.order_id = oi.order_id
        WHERE o.order_status = 'delivered'
        GROUP BY c.customer_state, DATE_TRUNC('month', o.order_date)
    )

    SELECT
        customer_state,
        month,
        revenue,
        LAG(revenue) OVER (PARTITION BY customer_state ORDER BY month) AS previous_month_revenue,
        ROUND(
            CASE 
                WHEN LAG(revenue) OVER (PARTITION BY customer_state ORDER BY month) = 0 THEN NULL
                ELSE 
                    (revenue - LAG(revenue) OVER (PARTITION BY customer_state ORDER BY month)) 
                    / LAG(revenue) OVER (PARTITION BY customer_state ORDER BY month)
            END,
            3
        ) AS growth_rate
    FROM monthly_revenue
    ORDER BY customer_state, month;
    """

    return pd.read_sql(query_mom_growth, engine)


In [73]:
geographic_sales_mom()

,customer_state,month,revenue,previous_month_revenue,growth_rate
0,AC,2017-01-01,723.14,NaN,NaN
1,AC,2017-02-01,597.40,723.14,-0.174
2,AC,2017-03-01,530.18,597.40,-0.113
3,AC,2017-04-01,1351.51,530.18,1.549
4,AC,2017-05-01,2371.73,1351.51,0.755
...,...,...,...,...,...
551,TO,2018-04-01,5370.51,5579.63,-0.037
552,TO,2018-05-01,3314.33,5370.51,-0.383
553,TO,2018-06-01,4987.03,3314.33,0.505
554,TO,2018-07-01,3795.09,4987.03,-0.239


In [87]:
def conversion_rate_by_city():
    """
    3. Taux de conversion par ville
    """
    query_conversion = """
    WITH total_clients AS (
        SELECT 
            customer_city,
            COUNT(DISTINCT customer_id) AS total_clients
        FROM customers
        GROUP BY customer_city
    ),

    clients_with_orders AS (
        SELECT 
            c.customer_city,
            COUNT(DISTINCT c.customer_id) AS ordering_clients
        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        WHERE o.order_status = 'delivered'
        GROUP BY c.customer_city
    )

    SELECT 
        t.customer_city,
        t.total_clients,
        COALESCE(o.ordering_clients, 0) AS ordering_clients,
        ROUND(CAST(COALESCE(o.ordering_clients, 0) AS NUMERIC) / t.total_clients,3) AS conversion_rate
    FROM total_clients t
    LEFT JOIN clients_with_orders o ON t.customer_city = o.customer_city
    WHERE t.total_clients >= 50
    ORDER BY conversion_rate DESC
    LIMIT 20;
    """

    return pd.read_sql(query_conversion, engine)


In [88]:
conversion_rate_by_city()

,customer_city,total_clients,ordering_clients,conversion_rate
0,leme,50,50,1.0
1,mesquita,76,76,1.0
2,itapevi,170,170,1.0
3,lajeado,51,51,1.0
4,jatai,54,54,1.0
5,mairipora,75,75,1.0
6,cruzeiro,56,56,1.0
7,cubatao,79,79,1.0
8,campo mourao,56,56,1.0
9,araguari,54,54,1.0


#### 3. Analyse temporelle et saisonnalité
```sql
-- Détectez les patterns saisonniers
SELECT
    EXTRACT(YEAR FROM order_date) as year,
    EXTRACT(MONTH FROM order_date) as month,
    EXTRACT(DOW FROM order_date) as day_of_week,
    COUNT(*) as order_count,
    SUM(price + freight_value) as total_revenue,
    AVG(price + freight_value) as avg_order_value
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE order_status = 'delivered'
GROUP BY ROLLUP(
    EXTRACT(YEAR FROM order_date),
    EXTRACT(MONTH FROM order_date),
    EXTRACT(DOW FROM order_date)
)
ORDER BY year, month, day_of_week;
```

---

In [92]:
def temporal_seasonality_analysis():
    """
    3. Analyse temporelle et saisonnalité : par année, mois, jour de la semaine
    """
    query = """
    SELECT
        EXTRACT(YEAR FROM o.order_purchase_timestamp) as year,
        EXTRACT(MONTH FROM o.order_purchase_timestamp) as month,
        EXTRACT(DOW FROM o.order_purchase_timestamp) as day_of_week,
        COUNT(*) as order_count,
        SUM(oi.price + oi.freight_value) as total_revenue,
        AVG(oi.price + oi.freight_value) as avg_order_value
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY ROLLUP(
        EXTRACT(YEAR FROM o.order_purchase_timestamp),
        EXTRACT(MONTH FROM o.order_purchase_timestamp),
        EXTRACT(DOW FROM o.order_purchase_timestamp)
    )
    ORDER BY year, month, day_of_week;
    """
    
    return pd.read_sql(query, engine)


## Partie 4 : Analyse prédictive avec SQL

### 🔮 Modèles simples en SQL

In [ ]:
#### 1. Prédiction de churn

def churn_prediction_sql():
    """
    Identifiez les clients à risque de churn

    Indicateurs :
    - Pas d'achat depuis X jours
    - Baisse de fréquence d'achat
    - Diminution du panier moyen
    - Changement de comportement géographique
    """

    churn_query = """
    WITH customer_activity AS (
        -- Calculez les métriques d'activité récente
        -- Comparez avec l'historique du client
        -- Scorez le risque de churn
    )

    SELECT
        customer_id,
        days_since_last_order,
        order_frequency_trend,
        monetary_trend,
        churn_risk_score,
        CASE
            WHEN churn_risk_score > 0.7 THEN 'High Risk'
            WHEN churn_risk_score > 0.4 THEN 'Medium Risk'
            ELSE 'Low Risk'
        END as churn_segment
    FROM customer_activity;
    """

    return pd.read_sql(churn_query, engine)


#### 2. Recommandations produits
```sql
-- Market Basket Analysis simplifié
WITH product_pairs AS (
    SELECT
        oi1.product_id as product_a,
        oi2.product_id as product_b,
        COUNT(*) as co_purchase_count
    FROM order_items oi1
    JOIN order_items oi2 ON oi1.order_id = oi2.order_id
    WHERE oi1.product_id != oi2.product_id
    GROUP BY oi1.product_id, oi2.product_id
    HAVING COUNT(*) >= 10  -- Seuil minimum
)

SELECT
    product_a,
    product_b,
    co_purchase_count,
    co_purchase_count::float / total_a.count as confidence
FROM product_pairs pp
JOIN (
    SELECT product_id, COUNT(*) as count
    FROM order_items
    GROUP BY product_id
) total_a ON pp.product_a = total_a.product_id
ORDER BY confidence DESC;
```

---

## Partie 5 : Intégration avec les APIs météo

### 🌤️ Croisement données météo/ventes
```python
def weather_sales_correlation():
    """
    Correlez vos données météo du Notebook 1 avec les ventes
    
    Hypothèses à tester :
    1. Les ventes de certaines catégories augmentent-elles avec la pluie ?
    2. Y a-t-il un impact de la température sur les achats ?
    3. Les livraisons sont-elles impactées par la météo ?
    """
    
    # Récupérez les données météo historiques pour les villes brésiliennes
    weather_query = """
    SELECT DISTINCT customer_city, customer_state
    FROM customers
    WHERE customer_state IN ('SP', 'RJ', 'MG', 'RS', 'SC')
    ORDER BY customer_city;
    """
    
    cities = pd.read_sql(weather_query, engine)
    
    # Intégrez avec l'API météo
    # Analysez les corrélations
    
    pass
```

### 📊 Dashboard géo-temporel
```python
def create_geotemporal_dashboard():
    """
    Créez un dashboard interactif combinant :
    - Carte des ventes par région
    - Évolution temporelle avec météo
    - Segments clients géolocalisés
    - Prédictions par zone géographique
    """
    pass
```

---
## 🏆 Livrables finaux

### 📈 Rapport d'analyse complet
1. **Segmentation RFM (Recency, Frenquency, Monetary) ** : 5-7 segments avec caractéristiques
2. **Analyse géographique**  : Performances par région + recommandations
3. **Prédictions churn** : Liste des clients à risque + actions
4. **Recommandations produits** : Top 10 des associations
5. **Impact météo** : Corrélations significatives identifiées

### 🚀 Pipeline automatisé
```python
def automated_analysis_pipeline():
    """
    Pipeline qui :
    1. Se connecte à la DB
    2. Exécute toutes les analyses
    3. Met à jour les segments clients
    4. Génère le rapport automatiquement
    5. Envoie des alertes si nécessaire
    """
    pass
```

---

## 🎓 Auto-évaluation

- [ ] **Connexion DB** : PostgreSQL fonctionnelle
- [ ] **Requêtes complexes** : JOINs, CTEs, fonctions analytiques
- [ ] **Gestion des erreurs** : Connexions robustes
- [ ] **Performance** : Requêtes optimisées avec index
- [ ] **Intégration** : SQL + Python + APIs
- [ ] **Insights actionables** : Recommandations business claires

### 🔗 Préparation au Notebook 3
Le prochain notebook portera sur NoSQL (MongoDB) avec des données de réseaux sociaux et d'IoT, en temps réel.

### 💡 Bases de données alternatives
- **PlanetScale** : MySQL serverless gratuit
- **MongoDB Atlas** : 512MB gratuit
- **FaunaDB** : Base multi-modèle gratuite
- **Hasura Cloud** : GraphQL + PostgreSQL